In [9]:
from datasets import load_dataset

eli5 = load_dataset("eli5", split="train_asks[:1000]")

Reusing dataset eli5 (/Users/ole/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa)


In [10]:
eli5 = eli5.flatten()
eli5 = eli5.train_test_split(test_size=0.2)
eli5["train"][0]


{'q_id': '2xreok',
 'title': 'Do we still use real mode for anything?',
 'selftext': 'Does this happen or everything is in virtual mode nowadays?',
 'document': '',
 'subreddit': 'askscience',
 'answers.a_id': ['cp31nef'],
 'answers.text': ['Can you explain?  [Real mode](_URL_0_) usually refers to the 16 bit mode of x86 processors.  In that sense, any x86 machine that is not using EFI will boot into real mode (and rapidly switch to 32 or 64 bit protected mode).'],
 'answers.score': [5],
 'title_urls.url': [],
 'selftext_urls.url': [],
 'answers_urls.url': ['http://en.wikipedia.org/wiki/Real_mode']}

In [72]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [73]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["answers.text"]], truncation=True)

In [74]:

tokenized_eli5 = eli5.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=eli5["train"].column_names,
)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [75]:
block_size = 128


def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [76]:
lm_dataset = tokenized_eli5.map(group_texts, batched=True, num_proc=4)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [100]:
tokenizer.decode (lm_dataset["train"][-1]["input_ids"])

" after they parse it, if you look at a rainbow, ROYGBIV, light at the red end of the spectrum is 'heavier', it moves more slowly. At the blue end it's 'lighter', it moves more quickly. Now they can cancel the OYG portion. They're making those beams hit your eye (or in this case, a detection grid) out of synch. So instead the red stream will get confused with other background events going on at the time, etc.\n\nIt's optical. The only reason time is mentioned is because there's a speed difference between the light frequencies."

In [101]:
tokenizer.decode(tokenized_eli5["train"][-1]["input_ids"])

'It\'s a fascinatingly simple idea. \n\nA light pulse has a certain bandwidth (i.e. it contains a bunch of different frequencies), which is larger the shorter the pulse is. \n\nIn this time-cloaking experiment, they "pull apart" this light pulse by accelerating the blue part of the spectrum and slowing down the red part of it, creating a spectral hole in the middle. This can be achieved by cleverly engineering an optical material\'s dispersion curve, i.e. the frequency-dependent refractive index experienced by the light pulse. \n\nNow that you have that spectral hole, you can put an event inside which is now "cloaked" (not really, but ok). Put the pulse together afterwards by reversing the initial dispersion and it will now look like that event in the middle never happened, the light pulse comes out exactly like it went in.\n\nIf you want more technical details, let me know. All due respect to PhysOrg... but it\'s a sensationalist bullshit title.\n\n"The fibre-based system steers light

In [108]:
from transformers import DataCollatorForLanguageModeling, DataCollator
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False, return_tensors="tf")

In [119]:
tf_train_set = lm_dataset["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    dummy_labels=True,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

In [127]:
tf_train_set

<PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(16, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(16, None), dtype=tf.int64, name=None), 'labels': TensorSpec(shape=(16, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(16,), dtype=tf.float32, name=None))>

In [115]:
lm_dataset['train']

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1698
})

In [128]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [130]:
import tensorflow as tf
from transformers import TFAutoModelForCausalLM
model = TFAutoModelForCausalLM.from_pretrained("gpt2")
model.compile(optimizer=optimizer)



2022-05-29 20:57:36.246956: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour, please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [133]:
for i in tf_train_set:
    print('FUck tensorflow')

FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck tensorflow
FUck ten

2022-05-29 21:01:38.893716: W tensorflow/core/framework/op_kernel.cc:1733] INVALID_ARGUMENT: ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length.
Traceback (most recent call last):

  File "/usr/local/lib/python3.9/site-packages/transformers/tokenization_utils_base.py", line 707, in convert_to_tensors
    tensor = as_tensor(value)

  File "/usr/local/lib/python3.9/site-packages/tensorflow/python/framework/constant_op.py", line 267, in constant
    return _constant_impl(value, dtype, shape, name, verify_shape=False,

  File "/usr/local/lib/python3.9/site-packages/tensorflow/python/framework/constant_op.py", line 279, in _constant_impl
    return _constant_eager_impl(ctx, value, dtype, shape, verify_shape)

  File "/usr/local/lib/python3.9/site-packages/tensorflow/python/framework/constant_op.py", line 304, in _constant_eager_impl
    t = convert_to_eager_tensor(valu

InvalidArgumentError: ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length.
Traceback (most recent call last):

  File "/usr/local/lib/python3.9/site-packages/transformers/tokenization_utils_base.py", line 707, in convert_to_tensors
    tensor = as_tensor(value)

  File "/usr/local/lib/python3.9/site-packages/tensorflow/python/framework/constant_op.py", line 267, in constant
    return _constant_impl(value, dtype, shape, name, verify_shape=False,

  File "/usr/local/lib/python3.9/site-packages/tensorflow/python/framework/constant_op.py", line 279, in _constant_impl
    return _constant_eager_impl(ctx, value, dtype, shape, verify_shape)

  File "/usr/local/lib/python3.9/site-packages/tensorflow/python/framework/constant_op.py", line 304, in _constant_eager_impl
    t = convert_to_eager_tensor(value, ctx, dtype)

  File "/usr/local/lib/python3.9/site-packages/tensorfl

In [131]:
model.fit(x=tf_train_set, epochs=3)

Epoch 1/3


2022-05-29 20:58:00.931613: W tensorflow/core/framework/op_kernel.cc:1733] INVALID_ARGUMENT: ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length.
Traceback (most recent call last):

  File "/usr/local/lib/python3.9/site-packages/transformers/tokenization_utils_base.py", line 707, in convert_to_tensors
    tensor = as_tensor(value)

  File "/usr/local/lib/python3.9/site-packages/tensorflow/python/framework/constant_op.py", line 267, in constant
    return _constant_impl(value, dtype, shape, name, verify_shape=False,

  File "/usr/local/lib/python3.9/site-packages/tensorflow/python/framework/constant_op.py", line 279, in _constant_impl
    return _constant_eager_impl(ctx, value, dtype, shape, verify_shape)

  File "/usr/local/lib/python3.9/site-packages/tensorflow/python/framework/constant_op.py", line 304, in _constant_eager_impl
    t = convert_to_eager_tensor(valu

  8/106 [=>............................] - ETA: 14:06 - loss: 3.9368

KeyboardInterrupt: 